# Parse ariel3_thunderstorm-noise

This is a quick Python example of parsing a tiny part of the relatively complicated Ariel3 satellite thunderstorm-noise data.

* Inspired by breqm76
* File format documentation at
https://docs.google.com/document/d/1GioN-8mch3GeruMrqG6qVLklQRDOh1TiARfYwI7BC4k/edit#heading=h.uydog7egy1sz
* Parsing the files at https://spdf.sci.gsfc.nasa.gov/pub/data/ariel/ariel3/langmuir_probe/plasma-frequency_electron-temperature_thunderstorm-noise/DATA2_DR002129_DR002129_20080804_091423/


In [1]:
def chunkstring(string, length):
    "Return the string, split up into fixed-width substrings of given length"
    return (string[0+i:length+i] for i in range(0, len(string), length))

def groupOf10(data, offset):
    "print the data as 10 sets of three 5-character numbers"    
    segment = data[offset:offset+10*3*5]
    print('\n'.join(chunkstring(' '.join(chunkstring(segment[:30*5],5)), 18)))

In [2]:
# File is encoded as EBCDIC (codec cp500)
with open('dr002129_f00001.phys.1', 'rt', encoding='cp500') as ariel3_thunderstorm_noise:
    d2129 = ariel3_thunderstorm_noise.read()

In [3]:
groupOf10(d2129, 0x11F02)

 -6.1  -9.0 -14.3 
 -6.2  -9.1 -14.4 
 -6.5  -9.1 -14.4 
 -6.8  -9.2 -14.5 
 -7.0  -9.3 -14.6 
 -7.2  -9.4 -14.7 
 -7.4  -9.5 -14.8 
 -7.6  -9.7 -14.9 
 -7.8  -9.8 -15.0 
 -8.0  -9.9 -15.1


In [4]:
groupOf10(d2129, 0x11FA6)

 -8.1 -10.0 -15.2 
 -8.2 -10.1 -15.3 
 -8.3 -10.2 -15.4 
 -8.4 -10.3 -15.5 
 -8.4 -10.4 -15.6 
 -8.5 -10.5 -15.7 
 -8.5 -10.6 -15.8 
 -8.6 -10.7 -15.9 
 -8.8 -10.8 -16.0 
 -9.0 -10.9 -16.1
